# SymPauli_expect

Expectation Value of a Pauli Sum is as the Trace of PauliMatrix * DensityMatrix

$ PauliSum = \sum_{i} \ p_i \ P_i $

$ DensityMatrix = \sum_{j k} \ l_j r_k \ | L_j > < R_k | $

Thus

$ ExpectationValue = Tr(\ \sum_{i j k} \ p_j l_j r_k \ P_i | L_j > < R_k | \ ) $

While $ P_i $ is a PauliOperator (matrix) hence $ P_i | L_j > $ is becoming a new vector, it then perform the outer product with $ < R_k | $

However, for finding the Trace, all you need to do is to sum the diagonal components of the matrix.

Therefore everything outside of diagonal path can be ignored.

Which leads to:

$ ExpectationValue = \sum_{i j k} \ p_j l_j r_k \ P_i | L_j > < R_k | \ [\ iff \ P_i | L_j > == R_k \ ] $

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
P = [0,1,2,3]
L = [0,1,0,1]
R = [0,1,1,0]

# First find out bitflip Pauli
bitflips = [ (p&1)^(p>>1) for p in P]
bitflips

# Then do P_i | L_j > to make new vector
PL = [p^l for p,l in zip(bitflips, L)]
PL

if any( pl^r for pl,r in zip(PL,R) ):
    print(f"diagonal > False")
else:
    print(f"diagonal > True")


[0, 1, 1, 0]

[0, 0, 1, 1]

diagonal > False


In [3]:
# Now we verify this by actually computing the matrix representation
import numpy as np
import numexpr as ne

P = np.array([0,1,2,3], dtype='uint8')
L = np.array([0,1,0,1], dtype='uint8')
R = np.array([0,1,1,0], dtype='uint8')

# First find out bitflip Pauli
bitflips = (P&1) ^ (P>>1)
bitflips

PL = bitflips ^ L
PL

# Since this a 4qubit system, lets expand the vector
ref = (2**np.arange(4))[::-1]
ref

PLvec = np.zeros(2**4, dtype='uint8')
PLval = ref[PL.astype(bool)]
PLval
PLval = PLval.sum()
PLval
PLvec[PLval] = 1
PLvec

Rvec = np.zeros(2**4, dtype='uint8')
Rval = ref[R.astype(bool)]
Rval
Rval = Rval.sum()
Rval
Rvec[Rval] = 1
Rvec

PLRmatrix = np.outer(PLvec, Rvec)
PLRmatrix

np.trace(PLRmatrix)

array([0, 1, 1, 0], dtype=uint8)

array([0, 0, 1, 1], dtype=uint8)

array([8, 4, 2, 1], dtype=int32)

array([2, 1], dtype=int32)

3

...

array([4, 2], dtype=int32)

6

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

0

# This proves the theory. So all you need to do is stack 3 for loops together.

Actually you can do 2 for loops.

Since the last step is to find if PL == R, we can do a hashtable check to accomplish this goal.

If there do no exist exact same state as PL in R set, automatically means that for every r in R we obtain zero.

In [4]:
# assume that you have already done encoding IXYZ into 0123
pauli_list = dict() # { binary pauli as tuple : coefficient }
left_state = dict() # { state as tuple : coefficient }
right_state = dict() # { state as tuple : coefficient }
expect_val = 0

# This is how you compute the coefficient for PL
# [ I X Y Z]
from operator import itemgetter as itg
from operator import mul
from functools import reduce
pl_coef = [None, None, [1j, -1j], [1, -1]]

for P, cP in pauli_list:
    #print(f"{P, cP = }")
    bitflips = [ (p&1)^(p>>1) for p in P ]

    for L, cL in left_state:
        #print(f"{L, cL = }")
        PL = tuple(p^l for p,l in zip(bitflips, L))

        # This also check whether there is a pl==r
        # At the sametime we can obtain the coefficient
        cR = right_state.get(PL, None)
        if cR is not None:
            expect_val +=  cP * cL * cR * reduce(mul, (pl_coef[p][l] for p,l in zip(P,L) if p>1), 1)
        #else:
        #    print(PL, 'unfound')

print(f"{expect_val = }")

expect_val = 0


# Benchmarking

In [5]:
# Benchmark for chain multiplication

import numpy as np

size = 100000
n_qubits = 12
np.random.seed = 100

testP = np.random.randint(0, 4, (size, n_qubits), dtype='uint8')
#testC = np.ones(size)
testC = np.random.normal(-1, 1, size)

from openfermion import QubitOperator as QO
control_data = []
for i,(paulis, coef) in enumerate(zip(testP, testC)):
    term = tuple( (i, chr(87+p)) for i,p in enumerate(paulis) if p )
    control_data.append(QO(term, coef))
#print(control_data[:10])

import numpy as np
experiment_data = []
for i, (coef, paulis) in enumerate(zip(testC, testP)):
    experiment_data.append([tuple(p for p in paulis), coef])
experiment_data = tuple(experiment_data)
#print(experiment_data[:10])

In [6]:
testP[:2]
print(experiment_data[0])

array([[0, 3, 0, 3, 1, 0, 3, 3, 3, 0, 1, 0],
       [3, 0, 0, 2, 0, 1, 2, 2, 3, 1, 0, 1]], dtype=uint8)

[(0, 3, 0, 3, 1, 0, 3, 3, 3, 0, 1, 0), -0.5801972024400194]


In [7]:
n_states = 32

denominator = 1/np.sqrt(2)**(n_states/2)
denominator

rand_state = np.random.randint(0, 2, (n_states, n_qubits), dtype='uint8').tolist()

pauli_list = experiment_data
left_state = tuple(
    (tuple(s), denominator)
    for s in rand_state
    )

count = 0
for row in left_state:
    print(row)
    count += 1
    if count > 10:
        break

right_state = dict(left_state)
left_state = dict(left_state)
pauli_list = dict(pauli_list)

0.0039062499999999957

((1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0), 0.0039062499999999957)
((0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0), 0.0039062499999999957)
((1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0), 0.0039062499999999957)
((0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0), 0.0039062499999999957)
((1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1), 0.0039062499999999957)
((0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1), 0.0039062499999999957)
((1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0), 0.0039062499999999957)
((0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0), 0.0039062499999999957)
((0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0), 0.0039062499999999957)
((0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0), 0.0039062499999999957)
((0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0), 0.0039062499999999957)


## PurePython Speed

In [8]:
%%time
valid_bitflips = set( tuple(l^r for l,r in zip(L,R)) for L in left_state for R in right_state )
print(len(valid_bitflips))
count = 0
for row in valid_bitflips:
    print(row)
    count += 1
    if count > 5:
        break

469
(0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0)
(1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0)
(1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0)
(1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1)
(1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0)
(1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0)
CPU times: total: 0 ns
Wall time: 2 ms


In [9]:
%%time

# assume that you have already done encoding IXYZ into 0123
#pauli_list = dict() # { binary pauli as tuple : coefficient }
#left_state = dict() # { state as tuple : coefficient }
#right_state = dict() # { state as tuple : coefficient }
expect_val = 0

# This is how you compute the coefficient for PL
# [ I X Y Z]
from operator import itemgetter as itg
from operator import mul
from functools import reduce
pl_coef = [None, None, [1j, -1j], [1, -1]]

for P, cP in pauli_list.items():
    #print(f"{P, cP = }")
    bitflips = tuple( (p&1)^(p>>1) for p in P )
    if bitflips in valid_bitflips:

        for L, cL in left_state.items():
            #print(f"{L, cL = }")
            PL = tuple(p^l for p,l in zip(bitflips, L))

            # This also check whether there is a pl==r
            # At the sametime we can obtain the coefficient
            cR = right_state.get(PL, None)
            if cR is not None:
                expect_val +=  cP * cL * cR * reduce(mul, (pl_coef[p][l] for p,l in zip(P,L) if p>1), 1)
            #else:
            #    print(PL, 'unfound')

print(f"{expect_val = }")

expect_val = (0.005508455687018302+0j)
CPU times: total: 4.44 s
Wall time: 4.44 s


## Binary Numpy Speed

In [10]:
def to_binary(arr):
    bin_ref = (2**np.arange(arr.shape[-1], dtype=np.uint64))[::-1]
    return np.dot(arr, bin_ref)

In [11]:
%%time
left_state_np = np.array(list(left_state.keys()), dtype='uint8')
right_state_np = np.array(list(right_state.keys()), dtype='uint8')
pauli_list_np = np.array(list(pauli_list.keys()), dtype='uint8')

CPU times: total: 46.9 ms
Wall time: 48 ms


In [12]:
%%time
# 1. XOR(left_state, right_state) to find the appropriate bitflips to make them matches
valid_bitflips = list(set( tuple(l^r for l,r in zip(L,R)) for L in left_state for R in right_state ))
valid_bitflips = set(to_binary(np.array(valid_bitflips, dtype='uint8')))
print(len(valid_bitflips))
count = 0
for row in valid_bitflips:
    print(row, bin(row))
    count += 1
    if count > 5:
        break

469
0 0b0
10 0b1010
2059 0b100000001011
2060 0b100000001100
2069 0b100000010101
2070 0b100000010110
CPU times: total: 0 ns
Wall time: 3 ms


In [13]:
%%time
# 2. Translate Pauli's bitflip behaviour to binary representation
bitflips_list = to_binary((pauli_list_np&1) ^ (pauli_list_np>>1))
count = 0
for row in bitflips_list:
    print(row, bin(row))
    count += 1
    if count > 5:
        break

130 0b10000010
373 0b101110101
43 0b101011
2626 0b101001000010
382 0b101111110
711 0b1011000111
CPU times: total: 0 ns
Wall time: 5 ms


In [14]:
%%time
# 3. So now we use binary repr as dict key
left_state_keys = to_binary(left_state_np)  # [int, int, int, int, ...]
right_state_keys = to_binary(right_state_np)
left_state_bin = dict(zip(left_state_keys, left_state.items()))
right_state_bin = dict(zip(right_state_keys, right_state.items()))

count = 0
for row in left_state_bin.items():
    print(row)
    count += 1
    if count > 3:
        break
count = 0
for row in right_state_bin.items():
    print(row)
    count += 1
    if count > 3:
        break

(2880, ((1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0), 0.0039062499999999957))
(1132, ((0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0), 0.0039062499999999957))
(2890, ((1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0), 0.0039062499999999957))
(1126, ((0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0), 0.0039062499999999957))
(2880, ((1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0), 0.0039062499999999957))
(1132, ((0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0), 0.0039062499999999957))
(2890, ((1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0), 0.0039062499999999957))
(1126, ((0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0), 0.0039062499999999957))
CPU times: total: 0 ns
Wall time: 0 ns


In [15]:
%%time

# 4. if and only if valid_bitflips
from collections import defaultdict as ddict

valid_pauli_list = { p: (bitflips, cp) for bitflips, (p, cp) in zip(bitflips_list, pauli_list.items()) if bitflips in valid_bitflips }

count = 0
for row in valid_pauli_list.items():
    print(row)
    count += 1
    if count > 3:
        break

((0, 3, 0, 3, 1, 0, 3, 3, 3, 0, 1, 0), (130, -0.5801972024400194))
((0, 0, 0, 3, 0, 3, 1, 0, 2, 0, 2, 1), (43, -1.823878357793116))
((2, 3, 2, 2, 3, 1, 3, 1, 3, 0, 1, 0), (2898, -1.0709011323849718))
((1, 1, 2, 1, 1, 0, 2, 1, 2, 0, 2, 1), (4027, -0.7589966122428184))
CPU times: total: 15.6 ms
Wall time: 19 ms


In [16]:
%%time

# assume that you have already done encoding IXYZ into 0123
#pauli_list = dict() # { binary pauli as tuple : coefficient }
#left_state = dict() # { state as tuple : coefficient }
#right_state = dict() # { state as tuple : coefficient }
expect_val = 0

# This is how you compute the coefficient for PL
# [ I X Y Z]
from operator import itemgetter as itg
from operator import mul
from functools import reduce
pl_coef = [None, None, [1j, -1j], [1, -1]]

for P, (binbitflips, cP) in valid_pauli_list.items():
    #print(f"{P, bitflips, cP = }")

    for binL, (L, cL) in left_state_bin.items():
        #print(f"{L, cL = }")
        binPL = binL ^ binbitflips

        # This also check whether there is a pl==r
        # At the sametime we can obtain the coefficient
        cR = right_state_bin.get(binPL, None)  # right_state = { binR: (R, cR) }
        if cR is not None:
            expect_val +=  cP * cL * cR[1] * reduce(mul, (pl_coef[p][l] for p,l in zip(P,L) if p>1), 1)

print(f"{expect_val = }")

expect_val = (0.005508455687018302+0j)
CPU times: total: 562 ms
Wall time: 566 ms


## Binary Numpy BatchCoef Speed

In [22]:
%%time

#Ver.BatchCoef
# assume that you have already done encoding IXYZ into 0123
#pauli_list = dict() # { binary pauli as tuple : coefficient }
#left_state = dict() # { state as tuple : coefficient }
#right_state = dict() # { state as tuple : coefficient }
expect_val = 0

# This is how you compute the coefficient for PL
# [ I X Y Z]
from operator import itemgetter as itg
from operator import mul
from functools import reduce, lru_cache
pl_coef = [None, None, [1j, -1j], [1, -1]]

@lru_cache(maxsize=None)
def _PaulizalCoef(_P, _L):
    return reduce(mul, (pl_coef[p][l] for p,l in zip(_P,_L) if p>1), 1)

def PaulizalCoef(P, L, pt=4):
    return reduce(mul, (_PaulizalCoef(P[k:k+pt], L[k:k+pt]) for k in range(0, len(P), pt)) , 1)

for P, (binbitflips, cP) in valid_pauli_list.items():
    #print(f"{P, bitflips, cP = }")

    for binL, (L, cL) in left_state_bin.items():
        #print(f"{L, cL = }")
        binPL = binL ^ binbitflips

        # This also check whether there is a pl==r
        # At the sametime we can obtain the coefficient
        cR = right_state_bin.get(binPL, None)  # right_state = { binR: (R, cR) }
        if cR is not None:
            expect_val +=  cP * cL * cR[1] * PaulizalCoef(P,L)

print(f"{expect_val = }")

expect_val = (0.005508455687018302+0j)
CPU times: total: 250 ms
Wall time: 253 ms


## Batch coef (?)

In [23]:
%%time
from cProfile import run

run("""
#Ver.BatchCoef
# assume that you have already done encoding IXYZ into 0123
#pauli_list = dict() # { binary pauli as tuple : coefficient }
#left_state = dict() # { state as tuple : coefficient }
#right_state = dict() # { state as tuple : coefficient }
expect_val = 0

# This is how you compute the coefficient for PL
# [ I X Y Z]
from operator import itemgetter as itg
from operator import mul
from functools import reduce
pl_coef = [None, None, [1j, -1j], [1, -1]]

@lru_cache(maxsize=None)
def _PaulizalCoef(_P, _L):
    return reduce(mul, (pl_coef[p][l] for p,l in zip(_P,_L) if p>1), 1)

def PaulizalCoef(P, L, pt=4):
    return reduce(mul, (_PaulizalCoef(P[k:k+pt], L[k:k+pt]) for k in range(0, len(P), pt)) , 1)

for P, (binbitflips, cP) in valid_pauli_list.items():
    #print(f"{P, bitflips, cP = }")

    for binL, (L, cL) in left_state_bin.items():
        #print(f"{L, cL = }")
        binPL = binL ^ binbitflips

        # This also check whether there is a pl==r
        # At the sametime we can obtain the coefficient
        cR = right_state_bin.get(binPL, None)  # right_state = { binR: (R, cR) }
        if cR is not None:
            expect_val +=  cP * cL * cR[1] * PaulizalCoef(P,L)

print(f"{expect_val = }")
""")

expect_val = (0.005508455687018302+0j)
         572079 function calls (568204 primitive calls) in 0.342 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.186    0.186    0.342    0.342 <string>:1(<module>)
     3875    0.003    0.000    0.028    0.000 <string>:16(_PaulizalCoef)
    11619    0.022    0.000    0.022    0.000 <string>:18(<genexpr>)
    24978    0.021    0.000    0.120    0.000 <string>:20(PaulizalCoef)
    99912    0.053    0.000    0.081    0.000 <string>:21(<genexpr>)
        1    0.000    0.000    0.000    0.000 functools.py:35(update_wrapper)
        1    0.000    0.000    0.000    0.000 functools.py:479(lru_cache)
        1    0.000    0.000    0.000    0.000 functools.py:518(decorating_function)
        1    0.000    0.000    0.000    0.000 iostream.py:202(schedule)
        2    0.000    0.000    0.000    0.000 iostream.py:429(_is_master_process)
        2    0.000    0.000    0.000    0.000

In [20]:
P = (2, 0, 2, 1, 3, 3, 2, 3, 3, 1, 0, 2, 3, 2)
L = (0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1)

pt = 4
for k in range(0, len(P), pt):
    end = k+pt
    p = P[k:end]
    l = L[k:end]
    print(p, l)

@lru_cache(maxsize=512)
def _PaulizalCoef(_P, _L):
    return reduce(mul, (pl_coef[p][l] for p,l in zip(_P,_L) if p>1), 1)

def PaulizalCoef(P, L, pt=2):
    return reduct(mul, (_PaulizalCoef(P[k:k+pt], L[k:k+pt]) for k in range(0, len(P), pt)) , 1)


(2, 0, 2, 1) (0, 1, 0, 0)
(3, 3, 2, 3) (1, 0, 1, 0)
(3, 1, 0, 2) (0, 0, 1, 1)
(3, 2) (0, 1)
